In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import essential packages 


import numpy as np
import pandas as pd
import os
import pickle
import re
from sklearn.datasets import load_files
import glob
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from tqdm.auto import tqdm

tqdm.pandas()

In [3]:
# Implementing pretrained word embeddings 



from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec


glove_file = datapath(os.path.abspath('glove.6B.50d.txt'))
tmp_file = get_tmpfile(os.path.abspath("test_word2vec.txt"))
converted_file = glove2word2vec(glove_file, tmp_file) 




In [4]:
# Loading the Glove embeddings in word2vec format 


glove_model = KeyedVectors.load_word2vec_format(os.path.abspath("test_word2vec.txt"))

In [5]:
glove_model["the"]  

array([ 4.1800e-01,  2.4968e-01, -4.1242e-01,  1.2170e-01,  3.4527e-01,
       -4.4457e-02, -4.9688e-01, -1.7862e-01, -6.6023e-04, -6.5660e-01,
        2.7843e-01, -1.4767e-01, -5.5677e-01,  1.4658e-01, -9.5095e-03,
        1.1658e-02,  1.0204e-01, -1.2792e-01, -8.4430e-01, -1.2181e-01,
       -1.6801e-02, -3.3279e-01, -1.5520e-01, -2.3131e-01, -1.9181e-01,
       -1.8823e+00, -7.6746e-01,  9.9051e-02, -4.2125e-01, -1.9526e-01,
        4.0071e+00, -1.8594e-01, -5.2287e-01, -3.1681e-01,  5.9213e-04,
        7.4449e-03,  1.7778e-01, -1.5897e-01,  1.2041e-02, -5.4223e-02,
       -2.9871e-01, -1.5749e-01, -3.4758e-01, -4.5637e-02, -4.4251e-01,
        1.8785e-01,  2.7849e-03, -1.8411e-01, -1.1514e-01, -7.8581e-01],
      dtype=float32)

In [6]:
# Stop word removal function 


from spacy.lang.en.stop_words import STOP_WORDS

def stop_word_remove(sentence):
    temp = [token for token in sentence.split() if token not in STOP_WORDS]
    return ' '.join(word for word in temp)

In [7]:
#  Function to read the papers from their paths  


def read_paper(path):
  f = open(path, 'r', encoding="utf-8")
  text = str(f.read())
  f.close()
  return text

In [8]:
def process_paper(text):

  # Removes unwanted characters, accounting for unicode characters
    text = re.sub("@&#", " ", text)
    text = re.sub("\n", " ", text)
    text = (text.encode('ascii', 'ignore')).decode("utf-8")

    # Extracting the highlights, body from the paper
    highlights = re.findall(r'HIGHLIGHTS(.*?)KEYPHRASES', text,  flags = re.I)[0]
    body_main = re.findall(r'(?<=\bABSTRACT\b).*', text, flags=re.I)[0]

    body = ' '.join(body_main.split())
    body = body.split('.')
  
    # Removing sentences that are too short or too long, as they wouldn't make apt summary text
    for i,x in enumerate(body):
        if (len(x.split())) < 3 or (len(x.split())) > 15: 
            body.pop(i)
    
    # Making a copy of the body, lowercasing body text, removing punctuations & extra spaces
    dummy_body = []
    for i in body:
        i= i.lower()
        i = re.sub('[^\w\s\d\.]','',i)   ###[^\w\s\d\.] '.'keep here
        dummy_body.append(i)

    # Making a copy of the highlights, lowercasing body text, removing punctuations & extra spaces
    dummy_highlights = highlights.lower()
    dummy_highlights = re.sub('[^\w\s\d]','',dummy_highlights) ###[^\w\s\d] '.' remove here
    dummy_highlights = ' '.join(dummy_highlights.split())
    
    # Removing stop words from body & highlights
    body_copy = []
    for x in dummy_body:
        body_copy.append(stop_word_remove(x))  # call to stop word function
        
    highlight_copy = []
    for x in dummy_highlights.split():
        highlight_copy.append(stop_word_remove(x)) # call to stop word function

    # Combing all of the highlights into one string    
    highlight_copy = " ".join(sentence for sentence in highlight_copy)
    highlight_copy = " ".join(highlight_copy.split())

    return body_main, body_copy, highlights, highlight_copy, body

In [9]:
from scipy.special import expit


# Function to calculate sentence Score
def document_score(body_copy, highlight_copy):
  # Getting word vectors for the body
  body_vectors = [] 
  for sent in body_copy:
      sent_vec = [] 
      for word in sent.split():
          if word in glove_model:
              sent_vec.append(glove_model[word])
          # If the word vector isn't there in the model
          # then use the vector of the word "Visual"
          else:
              sent_vec.append(glove_model["visual"])      
      body_vectors.append(sent_vec)

  # Getting word vectors for the highlights
  highlight_vectors = []
  for word in highlight_copy.split():
      if word in glove_model:
          highlight_vectors.append(glove_model[word])
      else:
          highlight_vectors.append(glove_model["visual"])

  # Finding the rouge score for each sentence by counting the # of common words
  # & dividing by length of sentence
  doc_score = []
  for sent in body_vectors:
      sent_score = 0
      for word in sent:
          for w in highlight_vectors:
              if (word == w).all():
                  sent_score+=1
                   
      if sent_score>0: 
        doc_score.append(expit(sent_score/len(sent)))
      else:
        doc_score.append(0)
  return doc_score

In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


def create_tokenizer(body_copy):
    tokenizer = Tokenizer(num_words = 600)
    tokenizer.fit_on_texts(body_copy) 
    t_len = len(tokenizer.word_index)
    body_copy_seq = tokenizer.texts_to_sequences(body_copy)
    body_copy_seq_pad = pad_sequences(body_copy_seq,maxlen=9)
    return body_copy_seq_pad

In [11]:
# Function to create datasets

def create_data(path):
  text = read_paper(path)
  body_main, body_copy, highlights, highlight_copy, body = process_paper(text)
  doc_score = document_score(body_copy, highlight_copy)
  body_copy_seq_pad = create_tokenizer(body_copy)
  x = body_copy_seq_pad
  y = np.array(doc_score)  
  y = y.reshape(y.shape[0],1)  
  return x, y, body

In [12]:
#Model Building
#bUILDING LSTM
from tensorflow.keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Flatten, Dropout
from tensorflow.keras.layers import Embedding
model = Sequential()


In [13]:

model.add(Embedding(input_dim=600+1, output_dim=256, input_length=9))    #num_words=600 ,max_len=9 in pad_sequence&x.shape[1]=9 
model.add(Dropout(0.2))  
model.add(LSTM(80, return_sequences = True))
model.add(LSTM(80, return_sequences = True))
model.add(LSTM(80, return_sequences = True))
model.add(LSTM(50, return_sequences = True))
model.add(LSTM(80))
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 9, 256)            153856    
                                                                 
 dropout (Dropout)           (None, 9, 256)            0         
                                                                 
 lstm (LSTM)                 (None, 9, 80)             107840    
                                                                 
 lstm_1 (LSTM)               (None, 9, 80)             51520     
                                                                 
 lstm_2 (LSTM)               (None, 9, 80)             51520     
                                                                 
 lstm_3 (LSTM)               (None, 9, 50)             26200     
                                                                 
 lstm_4 (LSTM)               (None, 80)                4

In [14]:
#  Creating a list of all file paths & partially fitting the model 


paths = glob.glob("Parsed_Papers/*.txt")
for i,path in enumerate(tqdm(paths[0:20])):
    x, y, body = create_data(path) 
    model.fit(x,y,epochs=30,batch_size = 64) 

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 1/30
4/4 [==============================] - 7s 26ms/step - loss: 0.2867
Epoch 2/30
4/4 [==============================] - 0s 24ms/step - loss: 0.2041
Epoch 3/30
4/4 [==============================] - 0s 24ms/step - loss: 0.1288
Epoch 4/30
4/4 [==============================] - 0s 28ms/step - loss: 0.1086
Epoch 5/30
4/4 [==============================] - 0s 25ms/step - loss: 0.1044
Epoch 6/30
4/4 [==============================] - 0s 29ms/step - loss: 0.0960
Epoch 7/30
4/4 [==============================] - 0s 26ms/step - loss: 0.0931
Epoch 8/30
4/4 [==============================] - 0s 26ms/step - loss: 0.0880
Epoch 9/30
4/4 [==============================] - 0s 26ms/step - loss: 0.0760
Epoch 10/30
4/4 [==============================] - 0s 27ms/step - loss: 0.0670
Epoch 11/30
4/4 [==============================] - 0s 28ms/step - loss: 0.0657
Epoch 12/30
4/4 [==============================] - 0s 25ms/step - loss: 0.0586
Epoch 13/30
4/4 [==============================] - 0s 28ms/st

3/3 [==============================] - 0s 23ms/step - loss: 0.0037
Epoch 30/30
3/3 [==============================] - 0s 23ms/step - loss: 0.0039
Epoch 1/30
3/3 [==============================] - 0s 21ms/step - loss: 0.0731
Epoch 2/30
3/3 [==============================] - 0s 22ms/step - loss: 0.0471
Epoch 3/30
3/3 [==============================] - 0s 21ms/step - loss: 0.0355
Epoch 4/30
3/3 [==============================] - 0s 21ms/step - loss: 0.0299
Epoch 5/30
3/3 [==============================] - 0s 21ms/step - loss: 0.0250
Epoch 6/30
3/3 [==============================] - 0s 20ms/step - loss: 0.0191
Epoch 7/30
3/3 [==============================] - 0s 20ms/step - loss: 0.0122
Epoch 8/30
3/3 [==============================] - 0s 23ms/step - loss: 0.0092
Epoch 9/30
3/3 [==============================] - 0s 20ms/step - loss: 0.0063
Epoch 10/30
3/3 [==============================] - 0s 20ms/step - loss: 0.0042
Epoch 11/30
3/3 [==============================] - 0s 22ms/step - loss: 0

3/3 [==============================] - 0s 44ms/step - loss: 0.0032
Epoch 28/30
3/3 [==============================] - 0s 48ms/step - loss: 0.0037
Epoch 29/30
3/3 [==============================] - 0s 42ms/step - loss: 0.0034
Epoch 30/30
3/3 [==============================] - 0s 29ms/step - loss: 0.0033
Epoch 1/30
4/4 [==============================] - 0s 25ms/step - loss: 0.0919
Epoch 2/30
4/4 [==============================] - 0s 24ms/step - loss: 0.0673
Epoch 3/30
4/4 [==============================] - 0s 24ms/step - loss: 0.0524
Epoch 4/30
4/4 [==============================] - 0s 44ms/step - loss: 0.0460
Epoch 5/30
4/4 [==============================] - 0s 39ms/step - loss: 0.0340
Epoch 6/30
4/4 [==============================] - 0s 39ms/step - loss: 0.0243
Epoch 7/30
4/4 [==============================] - 0s 42ms/step - loss: 0.0185
Epoch 8/30
4/4 [==============================] - 0s 35ms/step - loss: 0.0160
Epoch 9/30
4/4 [==============================] - 0s 36ms/step - loss: 0

<br>

Testing

In [27]:
# Testing Data 

t, b, body = create_data("Parsed_Papers/S0003687013000550.txt")


In [28]:
len(body)

111

In [29]:
# Predicting the top 4 summary sentences

c = model.predict(t)
c = c.reshape(c.shape[0],)
lst = pd.Series(c)
i = lst.nlargest(4)             
i = list(i.index)
i # Indices

4/4 [==============================] - 0s 6ms/step


[85, 41, 90, 21]

In [30]:
#  --- Predicted summary --- 

summary = []

for x in i:
    summary.append(body[x])
summary    

[' No such relationship was observed in the left hand',
 ' The measurement was performed first on the right and then the left hand',
 ', 2004; Solovieva etal',
 ' Dentists heavily utilize the thumb, index, and middle fingers in precision gripping in the right (or dominant) hand, while the left hand remains static for supportive work tasks']

In [32]:

Paper_Summary = ' '.join(summary)
Paper_Summary = ' '.join(Paper_Summary.split())
Paper_Summary = re.sub('[^\w\s\d\.]','',Paper_Summary)
Paper_Summary

'No such relationship was observed in the left hand The measurement was performed first on the right and then the left hand  2004 Solovieva etal Dentists heavily utilize the thumb index and middle fingers in precision gripping in the right or dominant hand while the left hand remains static for supportive work tasks'

In [31]:
# --- Highlights - which are gold standard summary of the paper --- 


text = read_paper("Parsed_Papers/S0003687013000550.txt")
body_main,_, highlights,_,_ = process_paper(text)
highlights = " ".join(highlights.split()).split(".")
highlights

['The relationship of task variation during dental work history with pinch grip strength among dentists was investigated',
 ' The dentists with the most hand-loading tasks were at an increased risk of low pinch grip strength',
 ' It is advisable among dentists to perform as diverse work tasks as possible to reduce the risk of decreased pinch grip strength',
 '']